In [1]:
!date

Tue Jul  2 17:17:32 UTC 2019


In [2]:
!nvidia-smi

Tue Jul  2 17:17:32 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    39W / 300W |      0MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [36]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.preprocessing import text, sequence
from keras.callbacks import LearningRateScheduler

from sklearn.model_selection import KFold, train_test_split
from random import sample
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, log_loss

In [4]:
#crawl-300d-2M.vec--> https://fasttext.cc/docs/en/english-vectors.html
#When pre-train embedding is helpful? https://www.aclweb.org/anthology/N18-2084
#There are many pretrained word embedding models: 
#fasttext, GloVe, Word2Vec, etc
#crawl-300d-2M.vec is trained from Common Crawl (a website that collects almost everything)
#it has 2 million words. Each word is represent by a vector of 300 dimensions.

#https://nlp.stanford.edu/projects/glove/
#GloVe is similar to crawl-300d-2M.vec. Probably, they use different algorithms.
#glove.840B.300d.zip: Common Crawl (840B tokens, 2.2M vocab, cased, 300d vectors, 2.03 GB download)
#tokens mean words. It has 2.2M different words and 840B (likely duplicated) words in total

#note that these two pre-trained models give 300d vectors.
EMBEDDING_FILES = [
    '../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec',
    '../input/glove840b300dtxt/glove.840B.300d.txt'
]

In [5]:
NUM_MODELS = 2
# the maximum number of different words to keep in the original texts
# 40_000 is a normal number
# 100_000 seems good too
MAX_FEATURES = 100000 

#this is the number of training sample to put in theo model each step
BATCH_SIZE = 512

#units parameters in Keras.layers.LSTM/cuDNNLSTM
#it it the dimension of the output vector of each LSTM cell.
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS

EPOCHS = 4

#we will convert each word in a comment_text to a number.
#So a comment_text is a list of number. How many numbers in this list?
#we want the length of this list is a constant -> MAX_LEN
MAX_LEN = 220

In [6]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    #each line in the file looks like 
    # apple 0.3 0.4 0.5 0.6 ...
    # that is a word followed by 300 float numbers

    with open(path) as f:
        #return dict(get_coefs(*line.strip().split(' ')) for line in f)
        return dict(get_coefs(*o.strip().split(" ")) for o in tqdm(f))

In [7]:
def build_matrix(word_index, path):
    #path: a path that contains embedding matrix
    #word_index is a dict of the form ('apple': 123, 'banana': 349, etc)
    # that means word_index[word] gives the index of the word
    # word_index was built from all commment_texts

    #we will construct an embedding_matrix for the words in word_index
    #using pre-trained embedding word vectors from 'path'

    embedding_index = load_embeddings(path)

    #embedding_matrix is a matrix of len(word_index)+1  x 300
    embedding_matrix = np.zeros((len(word_index) + 1, 300))

    # word_index is a dict. Each element is (word:i) where i is the index
    # of the word
    for word, i in word_index.items():
        try:
            #RHS is a vector of 300d
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            pass
    return embedding_matrix

In [8]:
def build_model(embedding_matrix, num_aux_targets):
   # a simpler version can be found here
   # https://www.tensorflow.org/tutorials/keras/basic_text_classification

   # Trainable params of the model: 1,671,687
   # Recall that the number of samples in train.csv is 1_804_874

    #words is a vector of MAX_LEN dimension
    words = Input(shape=(MAX_LEN,))

    #Embedding is the keras layer. We use the pre-trained embbeding_matrix
    # https://keras.io/layers/embeddings/
    # have to say that parameters in this layer are not trainable
    # x is a vector of 600 dimension
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)

    #*embedding_matrix.shape is a short way for 
    #input_dim = embedding_matrix.shape[0], output_dim  = embedding_matrix.shape[1]

    #here the author used pre-train embedding matrix.
    #instead of train from begining like in tensorflow example

    #https://stackoverflow.com/questions/50393666/how-to-understand-spatialdropout1d-and-when-to-use-it
    #x = SpatialDropout1D(0.25)(x)
    x = SpatialDropout1D(0.3)(x)

    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)

    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)

    hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])

    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    result = Dense(1, activation='sigmoid')(hidden)

    #num_aux_targets = 6 since y_aux_train has 6 columns
    aux_result = Dense(num_aux_targets, activation='sigmoid')(hidden)

    model = Model(inputs=words, outputs=[result, aux_result])

    #model.summary() will gives a good view of the model structure

    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(clipnorm=0.1),
        metrics=['accuracy'])



    return model

In [9]:
train = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

In [10]:
#take smaller data to use for prototyping
total_dataset_size = 6000
test_size = 1000
data_sample = train.sample(total_dataset_size, random_state=1)
del train
del test
train, test = train_test_split(data_sample, train_size=total_dataset_size-test_size, 
                               test_size=test_size, random_state=1, shuffle=True)
del data_sample

# inspect data_sample etc.

In [12]:
test.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
467525,817578,0.3,'The real health-care change we need? Strong l...,0.1,0.0,0.0,0.3,0.0,NaN,NaN,...,161016,rejected,0,0,1,1,0,0.0,0,10
1243616,5634256,0.0,"""It’s in the entire state’s best interest to b...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,357823,approved,0,0,2,8,0,0.0,0,4
426271,765291,0.0,"In case you haven't seen it, they showed this ...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,158952,approved,0,0,0,2,0,0.0,0,4
1085322,5443027,0.0,"Condolences to the families, friends and commu...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,345906,approved,0,0,0,0,0,0.0,10,4
621828,1003344,0.0,Agreed although the suggested amount of compen...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,168555,approved,0,0,0,1,0,0.0,0,4


In [13]:
#
#Take the columns 'comment_text' from train,
# then fillall NaN values by emtpy string '' (redundant)
x_train = train['comment_text'].fillna('').values

#if true, y_train[i] =1, if false, it is = 0
y_train = np.where(train['target'] >= 0.5, 1, 0)

y_aux_train = train[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]

#
#Take the columns 'comment_text' from test,
# then fillall NaN values by emtpy string '' (redundant)
x_test = test['comment_text'].fillna('').values

In [14]:
#eddie added
y_test = np.where(test['target'] >= 0.5, 1, 0)

In [15]:
# https://keras.io/preprocessing/text/
# tokenizer is a class with some method
tokenizer = text.Tokenizer(num_words=MAX_FEATURES)

#we apply method fit_on_texts of tokenizer on x_train and x_test
#it will initialize some parameters/attribute inside tokenizer
#https://github.com/keras-team/keras-preprocessing/blob/master/keras_preprocessing/text.py#L139
#https://github.com/keras-team/keras-preprocessing/blob/master/keras_preprocessing/text.py#L210

tokenizer.fit_on_texts(list(x_train) + list(x_test)) #QUESTION TODO remove x_test?
#for example, after fit_on_texts, we can type
#tokenizer.word_counts #give a OderedDict
#tokenizer.document_counts # an int
#tokenizer.word_index is a dict of words with correponding indices
#There are 410046 different words in all 'comment_text'
#len(tokenizer.word_index) == 410_046


#these words come from all 'comment_text' in training.csv and test.csv

#tokenizer.index_word: the inverse of tokenizer.word_index


#https://github.com/keras-team/keras-preprocessing/blob/master/keras_preprocessing/text.py#L267
#we will convert each word in a comment_text to a number.
#So a comment_text is a list of number.


x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)


#https://keras.io/preprocessing/sequence/
# https://github.com/keras-team/keras-preprocessing/blob/master/keras_preprocessing/sequence.py
#each comment_text is now a list of word
# we want the length of this list is a constant -> MAX_LEN
# if the list is longer, then we cut/trim it 
# if shorter, then we add/pad it with 0's at the beginning
x_train = sequence.pad_sequences(x_train, maxlen=MAX_LEN)
x_test = sequence.pad_sequences(x_test, maxlen=MAX_LEN)

In [16]:
# create an embedding_matrix 
#after this, embedding_matrix is a matrix of size
# len(tokenizer.word_index)+1   x 600
# we concatenate two matrices, 600 = 300+300
embedding_matrix = np.concatenate(
    [build_matrix(tokenizer.word_index, f) for f in EMBEDDING_FILES], axis=-1)
#embedding_matrix.shape 
#== (410047, 600)

#embedding_matrix[i] is a 600d vector representation of the word whose index is i
#embedding_matrix[10]
#tokenizer.index_word[10] == 'you'

2000001it [02:14, 14880.03it/s]
2196018it [02:36, 14057.28it/s]


# sample of train dataset for prototyping

In [17]:

# rand_indices = sample(range(len(x_train)),total_dataset_size)
# data_sample_x = x_train[rand_indices[0]:rand_indices[0]+1]
# data_sample_y = y_train[rand_indices[0]:rand_indices[0]+1]
# for i in range(1,len(rand_indices)):
#     data_sample_x = np.concatenate((data_sample_x,x_train[rand_indices[i]:rand_indices[i]+1]),
#                                    axis=0)
#     data_sample_y = np.concatenate((data_sample_y,y_train[rand_indices[i]:rand_indices[i]+1]),
#                                    axis=0)
    

In [18]:
kfold = KFold(5, random_state=1, shuffle=True)

In [38]:
%%time
val_acc = []
val_auroc1 = []
val_f1 = []
val_bin_cross = []
# enumerate splits
for i_train, i_test in kfold.split(x_train):
    #print('train: %s, test: %s' % (data[i_train], data[i_test]))

    model = None
    checkpoint_predictions = []
    weights = []



    #https://keras.io/callbacks/#learningratescheduler

    for model_idx in range(NUM_MODELS):
      # build the same models
        model = build_model(embedding_matrix, y_aux_train.shape[-1])
      # We train each model EPOCHS times
      # After each epoch, we reset learning rate (we are using Adam Optimizer)  
      # https://towardsdatascience.com/learning-rate-scheduler-d8a55747dd90

      # https://github.com/keras-team/keras/blob/master/keras/callbacks.py#L921
      # learningrate is the attribute 'lr' from Adam optimizer
      # see https://github.com/keras-team/keras/blob/master/keras/optimizers.py#L460
      # In Adam Optimizer, learning rate is changing after each batch
        for global_epoch in range(EPOCHS):
            model.fit(
                x_train[i_train], #2019JUL1 is i_train the spread out indices of the kfold?
                [y_train[i_train], y_aux_train.iloc[i_train,:]],
                batch_size=BATCH_SIZE,
                epochs=1,
                verbose=1,
                callbacks=[
                    #LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch), verbose = 1)
                    LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch))
                ]#,
                #validation=[,y_test[i_test]]  #2019JUL1: is this right?
            )
            #model.predict will give two outputs: main_output (target) and aux_output
            #we only take main_output
            checkpoint_predictions.append(model.predict(x_train[i_test], batch_size=2048)[0].flatten()) #is the normal way of 
                # of validating incompatible with this kernel because test is used in every iteration? maybe this is 
                # something to discuss with the team.
                # Change the training completely to something more normal? see how that performs?
                # Just ignore the checkpoints and weights and use the final model at the end for prediction on x_test?
                # Remove the learning rate scheduler?
            weights.append(2 ** global_epoch)
    predictions = np.average(checkpoint_predictions, weights=weights, axis=0)
    
    val_bin_cross.append(log_loss(y_train[i_test],predictions))
    
    #val_f1.append(f1_score(y_train[i_test],predictions))  #currently there is some bug
    
    #roc_auc
    val_auroc1.append(roc_auc_score(y_train[i_test],predictions))
    
    #accuracy
    predictions = np.where(predictions >= 0.5, 1, 0)
    val_acc.append(accuracy_score(y_train[i_test], predictions))

Epoch 1/1
4000/4000 [==============================] - 26s 7ms/step - loss: 0.6478 - dense_139_loss: 0.3383 - dense_140_loss: 0.3096 - dense_139_acc: 0.9230 - dense_140_acc: 0.8135
Epoch 1/1
4000/4000 [==============================] - 1s 286us/step - loss: 0.4223 - dense_139_loss: 0.2683 - dense_140_loss: 0.1540 - dense_139_acc: 0.9230 - dense_140_acc: 0.8557
Epoch 1/1
4000/4000 [==============================] - 1s 277us/step - loss: 0.4013 - dense_139_loss: 0.2573 - dense_140_loss: 0.1440 - dense_139_acc: 0.9230 - dense_140_acc: 0.8557
Epoch 1/1
4000/4000 [==============================] - 1s 280us/step - loss: 0.3912 - dense_139_loss: 0.2500 - dense_140_loss: 0.1412 - dense_139_acc: 0.9230 - dense_140_acc: 0.8557
Epoch 1/1
4000/4000 [==============================] - 27s 7ms/step - loss: 0.7205 - dense_143_loss: 0.3787 - dense_144_loss: 0.3418 - dense_143_acc: 0.8157 - dense_144_acc: 0.7915
Epoch 1/1
4000/4000 [==============================] - 1s 283us/step - loss: 0.4362 - dense_

In [40]:
#final score (average) of kfold cv of ensemble model
print(sum(val_acc)/float(len(val_acc)))

print(sum(val_auroc1)/float(len(val_auroc1)))
#print(sum(val_f1)/float(len(val_f1)))
print(sum(val_bin_cross)/float(len(val_bin_cross)))

0.9234
0.7679749474533772
0.2564531115626966


0.9315999999999999
0.8963434947479014
0.18568469874299043

In [35]:
val_acc

[0.7751783783783784,
 0.7841763827932201,
 0.7638681220095694,
 0.7717904759555495,
 0.7329965590453181]

In [24]:
#repeat the cell above for another model

In [45]:
%%time
val_acc = []
val_auroc2 = []
val_f1 = []
val_bin_cross = []
# enumerate splits
for i_train, i_test in kfold.split(x_train):
    #print('train: %s, test: %s' % (data[i_train], data[i_test]))

    model = None
    checkpoint_predictions = []
    weights = []

    
    model = build_model(embedding_matrix, y_aux_train.shape[-1])
    model.fit(
        x_train[i_train], #2019JUL1 is i_train the spread out indices of the kfold?
        [y_train[i_train], y_aux_train.iloc[i_train,:]],
        batch_size=BATCH_SIZE,
        epochs=8,
        verbose=1
    )
    predictions = model.predict(x_train[i_test], batch_size=2048)[0].flatten()
    
    
    
    val_bin_cross.append(log_loss(y_train[i_test],predictions))
    
    #val_f1.append(f1_score(y_train[i_test],predictions))  #currently there is some bug
    
    #roc_auc
    val_auroc2.append(roc_auc_score(y_train[i_test],predictions))
    
    #accuracy
    predictions = np.where(predictions >= 0.5, 1, 0)
    val_acc.append(accuracy_score(y_train[i_test], predictions))

Epoch 1/8
4000/4000 [==============================] - 25s 6ms/step - loss: 0.6765 - dense_179_loss: 0.3540 - dense_180_loss: 0.3225 - dense_179_acc: 0.9227 - dense_180_acc: 0.7762
Epoch 2/8
4000/4000 [==============================] - 1s 281us/step - loss: 0.4396 - dense_179_loss: 0.2843 - dense_180_loss: 0.1553 - dense_179_acc: 0.9230 - dense_180_acc: 0.8557
Epoch 3/8
4000/4000 [==============================] - 1s 277us/step - loss: 0.4097 - dense_179_loss: 0.2631 - dense_180_loss: 0.1466 - dense_179_acc: 0.9230 - dense_180_acc: 0.8557
Epoch 4/8
4000/4000 [==============================] - 1s 274us/step - loss: 0.3716 - dense_179_loss: 0.2340 - dense_180_loss: 0.1376 - dense_179_acc: 0.9230 - dense_180_acc: 0.8557
Epoch 5/8
4000/4000 [==============================] - 1s 281us/step - loss: 0.3550 - dense_179_loss: 0.2196 - dense_180_loss: 0.1354 - dense_179_acc: 0.9243 - dense_180_acc: 0.8554
Epoch 6/8
4000/4000 [==============================] - 1s 282us/step - loss: 0.3176 - dense

In [47]:
#final score (average) of kfold cv of ensemble model
print(sum(val_acc)/float(len(val_acc)))  #high is good

print(sum(val_auroc2)/float(len(val_auroc2)))  #high is good
#print(sum(val_f1)/float(len(val_f1)))
print(sum(val_bin_cross)/float(len(val_bin_cross)))   #low is good

0.9315999999999999
0.8963434947479014
0.18568469874299043


# training

In [48]:
# model = None
# checkpoint_predictions = []
# weights = []


# #https://keras.io/callbacks/#learningratescheduler

# for model_idx in range(NUM_MODELS):
#   # build the same models
#     model = build_model(embedding_matrix, y_aux_train.shape[-1])
#   # We train each model EPOCHS times
#   # After each epoch, we reset learning rate (we are using Adam Optimizer)  
#   # https://towardsdatascience.com/learning-rate-scheduler-d8a55747dd90

#   # https://github.com/keras-team/keras/blob/master/keras/callbacks.py#L921
#   # learningrate is the attribute 'lr' from Adam optimizer
#   # see https://github.com/keras-team/keras/blob/master/keras/optimizers.py#L460
#   # In Adam Optimizer, learning rate is changing after each batch
#     for global_epoch in range(EPOCHS):
#         model.fit(
#             x_train,
#             [y_train, y_aux_train],
#             batch_size=BATCH_SIZE,
#             epochs=1,
#             verbose=1,
#             callbacks=[
#                 #LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch), verbose = 1)
#                 LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch))
#             ]
#         )
#         #model.predict will give two outputs: main_output (target) and aux_output
#         #we only take main_output
#         checkpoint_predictions.append(model.predict(x_test, batch_size=2048)[0].flatten())
#         weights.append(2 ** global_epoch)

In [49]:
# indices = np.array([0,4,6])
# (y_aux_train.iloc[indices,:])

```
InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
```
Do i need to wait for the other gpu process to finish? this error message is suggesting that

# evaluate against test_size examples held out from train

# continue original

In [50]:
#take average (with weights) of predictions from two models
#predictions is an np.array
predictions = np.average(checkpoint_predictions, weights=weights, axis=0)

ZeroDivisionError: Weights sum to zero, can't be normalized

In [51]:
submission = pd.DataFrame.from_dict({
    'id': test['id'],
    'prediction': predictions
})

submission.to_csv('submission.csv', index=False)

In [52]:
!date

Tue Jul  2 19:18:03 UTC 2019
